<a href="https://colab.research.google.com/github/mxngjxa/learning_equality_recommendation/blob/main/minilm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#drive mounting for timestamping
from google.colab import drive
drive.mount('/content/drive')
import os


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
working_directory = "/content/drive/MyDrive/Colab Notebooks/DAS 435 Jacky/project_2/"

In [4]:
import sys, os
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn.functional as F
import torch.nn as nn
import math

os.environ["TOKENIZERS_PARALLELISM"] = "false"

def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(0)

MAX_LEN = 64

In [5]:
def adjust_lr(optimizer, epoch):
    if epoch >= 2:
        lr = 2e-5
    else:
        lr = 1e-6

    lr *= 4

    optimizer.param_groups[0]['lr'] = lr
    optimizer.param_groups[1]['lr'] = 100*lr

    return lr


def get_optimizer(net):
    params = [x[1] for x in filter(lambda kv: "backbone" in kv[0], net.named_parameters())]
    arc_weight = [x[1] for x in filter(lambda kv: "backbone" not in kv[0], net.named_parameters())]

    optimizer = torch.optim.Adam([{"params": params}, {"params": arc_weight}], lr=3e-4, betas=(0.9, 0.999),
                                 eps=1e-08)
    return optimizer

In [6]:
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, scale=30.0, margin=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.scale = scale

        self.ls_eps = ls_eps
        self.arc_weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.arc_weight)

        self.easy_margin = easy_margin
        self.set_margin(margin)

    def set_margin(self, margin):
        self.margin = margin
        self.cos_m = math.cos(margin)
        self.sin_m = math.sin(margin)

        self.th = nn.Parameter(torch.FloatTensor([math.cos(math.pi - margin)]), requires_grad=False)
        self.mm = nn.Parameter(torch.FloatTensor([math.sin(math.pi - margin) * margin]), requires_grad=False)

    def forward(self, input, label):
        cosine = F.linear(F.normalize(input), F.normalize(self.arc_weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m

        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)

        one_hot = label #torch.zeros(cosine.size(), device='cuda')
        #one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.scale

        return output

In [7]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForMaskedLM, AutoModel


class TrainingModel(nn.Module):
    def __init__(self, model, num_classes):
        super(TrainingModel, self).__init__()
        self.model = model
        self.arcface = ArcMarginProduct(768, num_classes, scale=20.0, margin=0.1)

    def forward(self, ids, mask, label):
        x = self.model(ids, mask)
        return self.arcface(x, label)


class VecModel(nn.Module):
    def __init__(self, model_name):
        super(VecModel, self).__init__()
        self.backbone = AutoModel.from_pretrained(model_name)

    def forward(self, ids, mask):
        out = self.backbone(ids, mask)[0]
        out = (out[:, 1:MAX_LEN//2, :]*mask[:, 1:MAX_LEN//2, None]).mean(axis=1)

        return F.normalize(out)

    def save(self, path):
        torch.save(self.state_dict(), path)

    def load(self, path):
        self.load_state_dict(torch.load(path, map_location='cpu'))

In [8]:
corr_df = pd.read_csv(working_directory + "data/correlations.csv")

corr_df["content_ids"] = corr_df["content_ids"].apply(lambda x: x.split())

corr_df = corr_df.explode('content_ids').reset_index(drop=True)
corr_df

,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d
1,t_00004da3a1b2,c_376c5a8eb028
2,t_00004da3a1b2,c_5bc0e1e2cba0
3,t_00004da3a1b2,c_76231f9d0b5e
4,t_00068291e9a4,c_639ea2ef9c95
...,...,...
279914,t_fff9e5407d13,c_d64037a72376
279915,t_fffbe1d5d43c,c_46f852a49c08
279916,t_fffbe1d5d43c,c_6659207b25d5
279917,t_fffe14f1be1e,c_cece166bad6a


In [9]:
topic_df = pd.read_csv(working_directory + "data/topics.csv")
topic_df["title"] = topic_df["title"].fillna("")

title_map = topic_df.set_index("id")["title"].to_dict()
parent_map = topic_df.set_index("id")["parent"].to_dict()

topic_df = topic_df[topic_df["has_content"]].reset_index(drop=True)
print(topic_df.shape)

topic_df["parent_title"] = topic_df["parent"].apply(lambda x: title_map.get(x, ""))
print(topic_df.shape)

topic_df["grandpa"] = topic_df["parent"].apply(lambda x: parent_map.get(x))
topic_df["grandpa_title"] = topic_df["grandpa"].apply(lambda x: title_map.get(x, ""))
print(topic_df.shape)

topic_df["ggrandpa"] = topic_df["grandpa"].apply(lambda x: parent_map.get(x))
topic_df["ggrandpa_title"] = topic_df["ggrandpa"].apply(lambda x: title_map.get(x, ""))
print(topic_df.shape)

topic_df["title"] = topic_df["title"] + " @ " + topic_df["parent_title"] + " @ " + topic_df["grandpa_title"] + " @ " + topic_df["ggrandpa_title"]

topic_df.head()

(61517, 9)
(61517, 10)
(61517, 12)
(61517, 14)


,id,title,description,channel,category,level,language,parent,has_content,parent_title,grandpa,grandpa_title,ggrandpa,ggrandpa_title
0,t_00004da3a1b2,Откриването на резисторите @ Открития и проект...,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True,Открития и проекти,t_c85886762db7,Физика,t_27a0a3c7ee9a,Наука
1,t_00068291e9a4,Entradas e saídas de uma função @ Álgebra: fun...,Entenda um pouco mais sobre funções.,8e286a,source,4,pt,t_d14b6c2a2b70,True,Álgebra: funções,t_461c9a8dd863,9º Ano,t_a6420bcff569,Matemática por ano (Alinhada à BNCC)
2,t_00069b63a70a,Transcripts @ Flow Charts: Logical Thinking? @...,NaN,6e3ba4,source,3,en,t_4054df11a74e,True,Flow Charts: Logical Thinking?,t_acbbd893e6af,Engineering,t_9c5f09334d83,MIT Blossoms
3,t_0006d41a73a8,Графики на експоненциални функции (Алгебра 2 н...,Научи повече за графиките на сложните показате...,000cf7,source,4,bg,t_e2452e21d252,True,Показателни и логаритмични функции,t_5f4cc8e02423,Алгебра (цялото съдържание),t_afd8db5e4f44,Математика
4,t_0008768bdee6,100 સુધીનો સરવાળો @ 100 સુધીના સરવાળા અને બાદબ...,37 અને 49 જેવી બે-અંકની સંખ્યાઓ ઉમેરતા શીખો.,5223e0,supplemental,4,gu,t_0da7a331d666,True,100 સુધીના સરવાળા અને બાદબાકી,t_d20fc01fdf38,પ્રારંભિક ગણિત,t_38bca5075c51,ધોરણ ૩


In [10]:
from sklearn.model_selection import KFold


N_FOLDS = 3

splits = KFold(N_FOLDS, shuffle=True, random_state=0).split(topic_df, topic_df["has_content"])

topic_df["fold"] = 0

for f, (train_ind, val_ind) in enumerate(splits):
    topic_df.loc[val_ind, "fold"] = f

topic_df["fold"].value_counts()

,count
fold,
0,20506
1,20506
2,20505


In [11]:
topic_df.loc[(topic_df["category"] == "source") | (topic_df["fold"] > 0), "fold"] = 1
topic_df["fold"].value_counts()

,count
fold,
1,53269
0,8248


In [12]:
le_topic = LabelEncoder()
topic_df["t"] = topic_df["title"].fillna("") + " | " + topic_df["description"].fillna("")

In [13]:
TEST_FOLD = 0

topic_df  = topic_df[topic_df["language"] != "ar"]

topic_df["sw"] = 1

topic_df_test = topic_df[topic_df["fold"] == TEST_FOLD].reset_index(drop=True)
topic_df_train = topic_df[topic_df["fold"] != TEST_FOLD].reset_index(drop=True)


topic_df_train.shape, topic_df_test.shape

<ipython-input-13-ffa394c734da>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topic_df["sw"] = 1


((50096, 17), (8248, 17))

In [14]:
content_df = pd.read_csv(working_directory + "data/content.csv")

content_df  = content_df[content_df["language"] != "ar"]

content_df.loc[content_df["title"] == content_df["description"], "description"] = None
content_df["description"].fillna(content_df["text"].fillna("").apply(lambda x: x[:256]), inplace=True)
content_df["t"] = content_df["title"].fillna("") + " | " + content_df["kind"].fillna("") + " | " + content_df["description"].fillna("")

print(content_df.shape)
content_df.head()

(146629, 9)


<ipython-input-14-4d0ff9db9a6c>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  content_df["description"].fillna(content_df["text"].fillna("").apply(lambda x: x[:256]), inplace=True)


,id,title,description,kind,text,language,copyright_holder,license,t
0,c_00002381196d,"Sumar números de varios dígitos: 48,029+233,930","Suma 48,029+233,930 mediante el algoritmo está...",video,NaN,es,NaN,NaN,"Sumar números de varios dígitos: 48,029+233,93..."
1,c_000087304a9e,Trovare i fattori di un numero,Sal trova i fattori di 120.\n\n,video,NaN,it,NaN,NaN,Trovare i fattori di un numero | video | Sal t...
2,c_0000ad142ddb,Sumar curvas de demanda,Cómo añadir curvas de demanda\n\n,video,NaN,es,NaN,NaN,Sumar curvas de demanda | video | Cómo añadir ...
3,c_0000c03adc8d,Nado de aproximação,Neste vídeo você vai aprender o nado de aproxi...,document,\nNado de aproximação\nSaber nadar nas ondas ...,pt,Sikana Education,CC BY-NC-ND,Nado de aproximação | document | Neste vídeo v...
4,c_00016694ea2a,geometry-m3-topic-a-overview.pdf,Estándares Comunes del Estado de Nueva York\n\...,document,Estándares Comunes del Estado de Nueva York\n\...,es,Engage NY,CC BY-NC-SA,geometry-m3-topic-a-overview.pdf | document | ...


In [15]:
train_contents = corr_df.merge(topic_df_train[["id"]], left_on="topic_id", right_on="id")
train_contents["sw"] = 1
train_contents["sw"] /= np.sqrt(train_contents.groupby("topic_id")["id"].transform("count"))

train_contents = train_contents.groupby("content_ids")["sw"].sum().reset_index()
train_contents["sw"] = train_contents["sw"].clip(0.1, 10)

train_contents.rename(columns={"content_ids": "id"}, inplace=True)
train_contents["is_train"] = True
train_contents["sw"] = train_contents.shape[0]*train_contents["sw"]/train_contents["sw"].sum()

train_contents["sw"] = 1
train_contents.shape

(133704, 3)

In [16]:
content_df = content_df.merge(train_contents, on="id", how="left")
content_df["is_train"].fillna(False, inplace=True)
content_df["sw"].fillna(1.0, inplace=True)

content_df["is_train"].mean()

<ipython-input-16-9923e0106987>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  content_df["is_train"].fillna(False, inplace=True)
<ipython-input-16-9923e0106987>:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  content_df["is_train"].fillna(False, inplace=True)
<ipython-input-16-9923e0106987>:3: FutureWarning

0.9115250052854483

In [17]:
content_df["label"] = 0
content_df.loc[content_df["is_train"], "label"] = le_topic.fit_transform(content_df.loc[content_df["is_train"], "t"])

In [18]:
#corr_df["label"] = le_topic.transform(corr_df["topic_id"])
corr_df = corr_df.merge(content_df[content_df["is_train"]][["id", "label"]].rename(columns={"id": "content_ids"}), on="content_ids")


content_labels = corr_df.groupby("topic_id").agg({"label": list}).reset_index()
content_labels = content_labels.rename(columns={"topic_id": "id"})
content_labels["label"] = content_labels["label"].apply(lambda x: list(set(x)))

topic_df_train = topic_df_train.merge(content_labels, on="id")

topic_df_train

,id,title,description,channel,category,level,language,parent,has_content,parent_title,grandpa,grandpa_title,ggrandpa,ggrandpa_title,fold,t,sw,label
0,t_00004da3a1b2,Откриването на резисторите @ Открития и проект...,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True,Открития и проекти,t_c85886762db7,Физика,t_27a0a3c7ee9a,Наука,1,Откриването на резисторите @ Открития и проект...,1,"[111544, 113249, 114120, 114624]"
1,t_00068291e9a4,Entradas e saídas de uma função @ Álgebra: fun...,Entenda um pouco mais sobre funções.,8e286a,source,4,pt,t_d14b6c2a2b70,True,Álgebra: funções,t_461c9a8dd863,9º Ano,t_a6420bcff569,Matemática por ano (Alinhada à BNCC),1,Entradas e saídas de uma função @ Álgebra: fun...,1,"[89691, 35260, 35259, 89692]"
2,t_00069b63a70a,Transcripts @ Flow Charts: Logical Thinking? @...,NaN,6e3ba4,source,3,en,t_4054df11a74e,True,Flow Charts: Logical Thinking?,t_acbbd893e6af,Engineering,t_9c5f09334d83,MIT Blossoms,1,Transcripts @ Flow Charts: Logical Thinking? @...,1,[48571]
3,t_0006d41a73a8,Графики на експоненциални функции (Алгебра 2 н...,Научи повече за графиките на сложните показате...,000cf7,source,4,bg,t_e2452e21d252,True,Показателни и логаритмични функции,t_5f4cc8e02423,Алгебра (цялото съдържание),t_afd8db5e4f44,Математика,1,Графики на експоненциални функции (Алгебра 2 н...,1,"[111488, 116870, 111475, 116276, 116277]"
4,t_0008768bdee6,100 સુધીનો સરવાળો @ 100 સુધીના સરવાળા અને બાદબ...,37 અને 49 જેવી બે-અંકની સંખ્યાઓ ઉમેરતા શીખો.,5223e0,supplemental,4,gu,t_0da7a331d666,True,100 સુધીના સરવાળા અને બાદબાકી,t_d20fc01fdf38,પ્રારંભિક ગણિત,t_38bca5075c51,ધોરણ ૩,1,100 સુધીનો સરવાળો @ 100 સુધીના સરવાળા અને બાદબ...,1,"[126376, 126369, 126548]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50091,t_fff7f2dd208b,Fatoração de polinômios encontrando fatores co...,Aprenda a fatorar utilizando fatores comuns.,8e286a,source,4,pt,t_28dfc9e80110,True,Álgebra: expressões algébricas,t_461c9a8dd863,9º Ano,t_a6420bcff569,Matemática por ano (Alinhada à BNCC),1,Fatoração de polinômios encontrando fatores co...,1,"[47584, 47552, 34508, 34509, 34510, 47572]"
50092,t_fff830472691,Scalar Projections @ Vector Analysis @ Analysi...,NaN,fef095,source,4,en,t_c75d6acecf78,True,Vector Analysis,t_de58fdbcad60,Analysis,t_9d8388491acd,Math,1,Scalar Projections @ Vector Analysis @ Analysi...,1,"[91761, 91762]"
50093,t_fff9e5407d13,NA_U06 - El periódico @ Lengua española @ PF (...,NaN,71fd51,supplemental,2,es,t_5bd8f6ae9f7d,True,Lengua española,t_3eca90fe0fe1,PF (Español),NaN,,1,NA_U06 - El periódico @ Lengua española @ PF (...,1,"[22630, 41353, 82122, 63443, 41684, 62933, 640..."
50094,t_fffbe1d5d43c,Inscribed shapes problem solving @ Miduara @ C...,Use properties of inscribed angles to prove pr...,0c929f,source,4,sw,t_50145b9bab3f,True,Miduara,t_c520677cef1d,Class 9 (India),t_4853606228df,Hisabati,1,Inscribed shapes problem solving @ Miduara @ C...,1,"[18867, 85597]"


In [19]:
content_df["label"] = content_df["label"].apply(lambda x: [x])

topic_df_test["label"] = 0
topic_df_test["label"] = topic_df_test["label"].apply(lambda x: [x])

In [20]:
len(le_topic.classes_), content_df["is_train"].sum()

(131987, 133656)

In [21]:
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, add_prefix_space=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [22]:
NW = 8
BS = 64

class LECRDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=512, aug=False):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.mask_token = self.tokenizer.convert_tokens_to_ids(self.tokenizer.mask_token)
        self.aug = aug
        self.num_classes = len(le_topic.classes_)

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]
        encoding = self.tokenizer(str(row.t), return_offsets_mapping=True,
                                  padding='max_length', truncation=True, max_length=self.max_len)

        encoding = {key: torch.as_tensor(val) for key, val in encoding.items()}

        if self.aug:
            ix = torch.rand(size=(self.max_len,)) < 0.15
            encoding["input_ids"][ix] = self.mask_token

        target = torch.zeros(self.num_classes, dtype=torch.float32)
        for l in row.label:
            target[l] = 1/len(row.label)

        return encoding, target, torch.FloatTensor([row.sw])






vec_model = VecModel(MODEL_NAME)
model = TrainingModel(vec_model, num_classes=len(le_topic.classes_))
model = model.cuda()

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

In [23]:
from sklearn.neighbors import NearestNeighbors


def to_vec(model, ds):
    val_loader = DataLoader(ds, batch_size=BS, shuffle=False, num_workers=NW,
                             pin_memory=False, drop_last=False)

    model.eval()

    tbar = tqdm(val_loader, file=sys.stdout)

    vectors = []

    with torch.no_grad():
        for idx, (data, target, _) in enumerate(tbar):
            ids, mask, target = data["input_ids"].cuda(), data["attention_mask"].cuda(), target.cuda()

            vec = model.model(ids, mask)
            vectors.append(vec.detach().cpu().numpy())

    V = np.concatenate(vectors)

    return V



def get_matches(V_topic, V_content, topic_ids, content_ids, n_neighbors=5):

    neighbors_model = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine', n_jobs=-1)
    neighbors_model.fit(V_content)
    dists, indices = neighbors_model.kneighbors(V_topic)

    res_df = pd.DataFrame({"topic_id": np.repeat(topic_ids, n_neighbors),
                           'content_id': content_ids[indices.ravel()],
                           'vec_dist': dists.ravel()
                          })
    return res_df


def f2(recall, precision):
    return 5*precision*recall/(4*precision + recall)

def metric(pred_df):
    scores = np.zeros(pred_df.shape[0])

    for i, (gt, pred) in enumerate(zip(pred_df["content_ids"].values, pred_df["pred"].values)):
        hits = 0

        gt, pred = gt.split(), pred.split()

        for l in gt:
            if l in pred:
                hits += 1

        if hits > 0:
            recall = hits / len(gt)
            precision = hits / len(pred)

            scores[i] = f2(recall, precision)

    return scores.mean()


def evaluate(topic_df, content_df):
    topic_ds = LECRDataset(topic_df, tokenizer, MAX_LEN, aug=False)
    content_ds = LECRDataset(content_df, tokenizer, MAX_LEN, aug=False)


    V_topic = to_vec(model, topic_ds)
    V_content = to_vec(model, content_ds)

    res_df = get_matches(V_topic, V_content, topic_df["id"].values, content_df["id"].values)

    res_df["rank"] = res_df.groupby("topic_id")["vec_dist"].rank(method="first", ascending=True)

    pred_df = res_df.copy() #res_df[(res_df["rank"] == 1) | (res_df["vec_dist"] < 0.1)]
    pred_df = pred_df.groupby("topic_id")["content_id"].apply(lambda x: " ".join(list(x)))
    pred_df = pred_df.reset_index().rename(columns={"content_id": 'pred'})
    pred_df = pred_df.merge(pd.read_csv(working_directory + "data/correlations.csv"), on="topic_id")

    return metric(pred_df)

In [24]:
cls_df = content_df[content_df["is_train"]]
cls_df["ix"] = le_topic.transform(cls_df["t"])
cls_df = cls_df[["ix", "t"]].drop_duplicates().sort_values("ix").reset_index(drop=True)

cls_df["label"] = cls_df["ix"].apply(lambda x: [x])
cls_df["sw"] = 1

ds = LECRDataset(cls_df, tokenizer, MAX_LEN, aug=False)

V_cls = to_vec(model, ds)

<ipython-input-24-b09baf888b03>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cls_df["ix"] = le_topic.transform(cls_df["t"])
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


100%|██████████| 2063/2063 [04:02<00:00,  8.52it/s]


In [25]:
model.arcface.arc_weight = nn.Parameter(torch.FloatTensor(V_cls))
model.cuda()

TrainingModel(
  (model): VecModel(
    (backbone): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(250037, 384, padding_idx=0)
        (position_embeddings): Embedding(512, 384)
        (token_type_embeddings): Embedding(2, 384)
        (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=384, out_features=384, bias=True)
                (key): Linear(in_features=384, out_features=384, bias=True)
                (value): Linear(in_features=384, out_features=384, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=384, out_features=384, bias=True)
 

In [28]:
def cross_entropy(pred, soft_targets, sw):
    logsoftmax = nn.LogSoftmax(dim=1)
    return torch.mean(torch.sum(- soft_targets * logsoftmax(pred), 1)*sw)


def train(model, epochs=2, n_batches=1, dynamic_margin=True):
    train_df = pd.concat([topic_df_train, content_df[content_df["is_train"]]], ignore_index=True)
    train_ds = LECRDataset(train_df, tokenizer, MAX_LEN, aug=False)

    train_loader = DataLoader(train_ds, batch_size=BS, shuffle=True, num_workers=NW,
                               pin_memory=False, drop_last=True)

    optimizer = get_optimizer(model)

    scaler = torch.cuda.amp.GradScaler()

    for e in range(epochs):
        if dynamic_margin:
            model.arcface.set_margin(0.1 + e*0.02)
        model.cuda()
        model.train()
        tbar = tqdm(train_loader, file=sys.stdout)

        lr = adjust_lr(optimizer, e)

        loss_list, acc_list = [], []

        optimizer.zero_grad()
        for idx, (data, label, sw) in enumerate(tbar):
            ids, mask, label = data["input_ids"].cuda(), data["attention_mask"].cuda(), label.cuda()

            with torch.cuda.amp.autocast():
                preds = model(ids, mask, label)
                loss = cross_entropy(preds, label, sw.cuda())

            loss = loss / n_batches
            scaler.scale(loss).backward()

            if ((idx + 1) % n_batches) == 0:
                scaler.unscale_(optimizer)
                nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            acc = (preds.softmax(dim=1)*(label > 0)).sum(axis=1).mean()

            loss_list.append(loss.detach().cpu().item())
            acc_list.append(acc.detach().cpu().item())
            avg_loss = np.round(n_batches * np.mean(loss_list), 4)
            avg_acc = np.round(np.mean(acc_list), 2)

            tbar.set_description(f"Epoch {e + 1} Loss: {avg_loss} Acc: {avg_acc} lr: {lr}")

        torch.save(model.state_dict(), working_directory + "models/tmp.pth")
        log_text = f"Epoch {e+1}\n"
        logfile = open(working_directory + f"logs/log.txt", 'a')
        logfile.write(log_text)
        logfile.close()

        val_score = evaluate(topic_df_test, content_df)
        print("Val score:", val_score)
        print()


torch.save(model.state_dict(), working_directory + "models/tmp.pth")
log_text = f"Epoch 0\n"
logfile = open(working_directory + f"logs/log.txt", 'a')
logfile.write(log_text)
logfile.close()

train(model, epochs=20)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-28-a416b213071f>:15: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


  0%|          | 0/2871 [00:00<?, ?it/s]

<ipython-input-28-a416b213071f>:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


100%|██████████| 2292/2292 [04:33<00:00,  8.37it/s]
Val score: 0.2229646722911377

  0%|          | 0/2871 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-28-a416b213071f>:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


...

100%|██████████| 2292/2292 [04:27<00:00,  8.55it/s]
Val score: 0.5074383602104312

  0%|          | 0/2871 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-28-a416b213071f>:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


100%|██████████| 2292/2292 [04:31<00:00,  8.45it/s]
Val score: 0.5106344887571261

  0%|          | 0/2871 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-28-a416b213071f>:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


100%|██████████| 2292/2292 [04:33<00:00,  8.39it/s]
Val score: 0.5113136719726668



In [ ]:
def adjust_lr(optimizer, epoch):
    if epoch >= 1:
        lr = 8e-6
    else:
        lr = 2e-5

    optimizer.param_groups[0]['lr'] = lr
    optimizer.param_groups[1]['lr'] = 100*lr

    return lr

model.arcface.set_margin(0.5)
train(model, epochs=2, dynamic_margin=False)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-28-a416b213071f>:15: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


  0%|          | 0/2871 [00:00<?, ?it/s]

<ipython-input-28-a416b213071f>:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 1 Loss: 1.0129 Acc: 0.58 lr: 2e-05:  82%|████████▏ | 2346/2871 [08:38<01:44,  5.01it/s]

In [ ]:
model.model.save(working_directory + "models/vec_model_v36_mini.pth")

In [ ]:
topic_ds = LECRDataset(topic_df_test, tokenizer, MAX_LEN, aug=False)
content_ds = LECRDataset(content_df, tokenizer, MAX_LEN, aug=False)


V_topic = to_vec(model, topic_ds)
V_content = to_vec(model, content_ds)

res_df = get_matches(V_topic, V_content, topic_df_test["id"].values, content_df["id"].values,
                    n_neighbors=20)

In [ ]:
def best_case_metric(pred_df):
    scores = np.zeros(pred_df.shape[0])

    for i, (gt, pred) in enumerate(zip(pred_df["content_ids"].values, pred_df["pred"].values)):
        hits = 0

        gt, pred = gt.split(), pred.split()

        for l in gt:
            if l in pred:
                hits += 1

        if hits > 0:
            recall = hits / len(gt)
            precision = 1 #hits / len(pred)

            scores[i] = f2(recall, precision)

    return scores.mean()


pred_df = res_df.groupby("topic_id")["content_id"].apply(lambda x: " ".join(list(x)))
pred_df = pred_df.reset_index().rename(columns={"content_id": 'pred'})
pred_df = pred_df.merge(pd.read_csv(working_directory + "data/correlations.csv"), on="topic_id")

best_case_metric(pred_df)

In [ ]:
res_df["rank"] = res_df.groupby("topic_id")["vec_dist"].rank(method="dense")


pred_df = res_df[(res_df["rank"] <= 2) | (res_df["vec_dist"] < 0.3)]
pred_df = pred_df.groupby("topic_id")["content_id"].apply(lambda x: " ".join(list(x)))
pred_df = pred_df.reset_index().rename(columns={"content_id": 'pred'})
pred_df = pred_df.merge(pd.read_csv(working_directory + "data/correlations.csv"), on="topic_id")

metric(pred_df)

In [ ]:
res_df["vec_dist"].hist(bins=50)

In [ ]:
x = res_df.merge(content_df[["id", "language"]], left_on="content_id", right_on="id")
x = x.merge(topic_df[["id", "language"]], left_on="topic_id", right_on="id")
x

In [ ]:
x["same_lang"] = x["language_x"] == x["language_y"]
x["same_lang"].mean(), x[x["vec_dist"] < 0.5]["same_lang"].mean()

In [ ]:
pred_df = x[(x["rank"] <= 1) | (x["same_lang"] & (x["vec_dist"] < 0.45))]
pred_df = pred_df.groupby("topic_id")["content_id"].apply(lambda x: " ".join(list(x)))
pred_df = pred_df.reset_index().rename(columns={"content_id": 'pred'})
pred_df = pred_df.merge(pd.read_csv(working_directory + "data/correlations.csv"), on="topic_id")

metric(pred_df)

In [ ]:
res_df = get_matches(V_topic, V_content, topic_df_test["id"].values, content_df["id"].values,
                    n_neighbors=50)

pred_df = res_df.groupby("topic_id")["content_id"].apply(lambda x: " ".join(list(x)))
pred_df = pred_df.reset_index().rename(columns={"content_id": 'pred'})
pred_df = pred_df.merge(pd.read_csv(working_directory + "data/correlations.csv"), on="topic_id")

best_case_metric(pred_df)